In [1]:
import os
import grant_query
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [2]:
KG_PATH = "../kg/data"
DERIVED_DATA_PATH = "../derived_data"

In [3]:
START_FISCAL_YEAR = 2021 # starts Oct. 1, 2020

In [4]:
projects = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "radx-projects.csv"))
projects.query("researchInitiative == 'RADx-rad'", inplace=True)

In [5]:
# reformat principal investigator name for improved name matching (lastname firstname)
projects["project_pi"] = projects["studyInvestigator"].apply(lambda x: x.rsplit(" ", 1)[1] + " " + x.rsplit(" ", 1)[0])

In [6]:
print(f"Number of projects {projects.shape[0]}")
projects.head()

Number of projects 50


,researchInitiative,dbgapAccession,projectNum,coreProjectNum,projectSerialNum,studyInvestigator,studyTitle,subProject,project_pi
0,RADx-rad,phs002744.v1.p1,U01AA029345,U01AA029345,AA029345,Khalid Salaita,Rapid Acceleration of Diagnostics - Radical (RADx-rad): Rolosense: An Innovative Platform for Automatic Mobile Phone Readout of Active SARS-CoV-2,Automatic Detection & Tracing,Salaita Khalid
12,RADx-rad,phs002778.v1.p1,1U18TR003793-01,U18TR003793,TR003793,Shannon Stott,Rapid Acceleration of Diagnostics - Radical (RADx-rad): Microfluidic Isolation and Characterization of SARS-CoV-2 and Virus Related Exosomes,Exosome,Stott Shannon
26,RADx-rad,phs002604.v1.p1,1U01DA053949-01,U01DA053949,DA053949,Anne-Catrin Uhlemann,Rapid Acceleration of Diagnostics - RADICAL (RADx-rad): Tracking the COVID-19 Epidemic in Sewage (TRACES),Wastewater,Uhlemann Anne-Catrin
27,RADx-rad,phs002583.v1.p1,5U01HL150852-02,U01HL150852,HL150852,Edward P DeMauro,Rapid Acceleration of Diagnostics - Radical (RADx-rad): A Rapid Breathalyzer Diagnostics Platform for COVID-19,Novel Biosensing and VOC,DeMauro Edward P
28,RADx-rad,phs002603.v1.p1,3R33HD105594-03S1,R33HD105594,HD105594,Audrey Odom-John,Rapid Acceleration of Diagnostics - Radical (RADx-rad): Diagnosis of MIS-C in Febrile Children,PreVAIL kIds,Odom-John Audrey


In [7]:
project_num = list(projects["coreProjectNum"].unique())

In [8]:
pis = grant_query.get_principal_investigators(list(project_num))
pis["fiscalYear"] = pis["fiscalYear"].astype(int)
pis = pis[(pis["fiscalYear"] >= START_FISCAL_YEAR) | pis["fiscalYear"].isna()]
#pis["appl_id"] = pis["appl_id"].astype(str)
#print("Number of PIs:", pis["profileId"].nunique())
pis.head()

,profileId,coreProjectNum,projectSerialNum,isContactPi,fiscalYear,name,fullName,firstName,middleName,lastName
0,1877373,U01DC019578,DC019578,True,2023,Dalton PH,Pamela Helen Dalton,Pamela,Helen,Dalton
1,16392635,U01DC019578,DC019578,False,2023,Parma V,Valentina Parma,Valentina,,Parma
2,9436729,R01MD016526,MD016526,True,2023,Dillard DA,Denise A Dillard,Denise,A,Dillard
3,10320851,R33HD105594,HD105594,False,2023,Bassiri H,Hamid Bassiri,Hamid,,Bassiri
4,9198778,R33HD105594,HD105594,True,2023,Odom John AR,Audrey Ragan Odom John,Audrey,Ragan,Odom John


In [9]:
pis["profilId"] = pis["profileId"].astype(str)
print(pis["profileId"].nunique())
profile_ids = list(pis["profileId"].unique())

99


In [10]:
organizations = grant_query.get_organizations(profile_ids)
organizations.rename(columns={"profile_id": "profileId"}, inplace=True)
print(organizations["profileId"].nunique())
organizations.sort_values("profileId", inplace=True)

83


In [11]:
organizations

,profileId,first_name,middle_name,last_name,is_contact_pi,full_name,title,appl_id,fiscal_year,organization.org_name,organization.org_city,organization.org_zipcode,organization.org_state,organization.org_country,organization.primary_duns,organization.primary_uei
84,10046004,Craig,G,Rusin,True,Craig G Rusin,ASSISTANT PROFESSOR (TENURE TRACK),10418753,2022,BAYLOR COLLEGE OF MEDICINE,HOUSTON,770303411,TX,UNITED STATES,051113330,FXKMA43NTV21
97,10129440,Rachel,Todd,Noble,True,Rachel Todd Noble,PROFESSOR,10320994,2022,UNIV OF NORTH CAROLINA CHAPEL HILL,CHAPEL HILL,275995023,NC,UNITED STATES,608195277,D3LHU66KBLD5
105,10311819,Scott,M,Berry,True,Scott M Berry,ASSOCIATE PROFESSOR,10320997,2022,UNIVERSITY OF KENTUCKY,LEXINGTON,405260001,KY,UNITED STATES,939017877,H1HYA8Z1NTM5
154,10320851,Hamid,,Bassiri,True,Hamid Bassiri,ASSISTANT PROFESSOR,8894464,2015,CHILDREN'S HOSP OF PHILADELPHIA,PHILADELPHIA,191462305,PA,UNITED STATES,073757627,G7MQPLSUX1L4
80,10349485,Samarjit,,Das,True,Samarjit Das,ASSISTANT PROFESSOR,10666956,2022,JOHNS HOPKINS UNIVERSITY,BALTIMORE,212182680,MD,UNITED STATES,001910777,FTMTDMBR29C7
89,10379550,Christopher,Edward,Mason,True,Christopher Edward Mason,ASSISTANT PROFESSOR,10378056,2022,WEILL MEDICAL COLL OF CORNELL UNIV,NEW YORK,100654805,NY,UNITED STATES,060217502,YNT8TCJH8FQ8
147,10450719,Eliah,S,Aronoff-Spencer,True,Eliah S Aronoff-Spencer,ASST ADJUNCT PROFESSOR (PROPOSED),9240688,2017,"UNIVERSITY OF CALIFORNIA, SAN DIEGO",LA JOLLA,920930621,CA,UNITED STATES,804355790,UYTTZT6G9DT1
94,10807170,Xiao-An,,Fu,True,Xiao-An Fu,,10320985,2022,UNIVERSITY OF LOUISVILLE,LOUISVILLE,402920001,KY,UNITED STATES,057588857,E1KJM4T54MK6
76,10833925,Otakuye,,Conroy-Ben,True,Otakuye Conroy-Ben,,10700056,2023,"INTER TRIBAL COUNCIL OF ARIZONA, INC.",PHOENIX,850041448,AZ,UNITED STATES,089993752,QVBEM11FYF47
98,11165737,Helena,,Solo-Gabriele,True,Helena Solo-Gabriele,PROFESSOR AND ASSOCIATE DEAN FOR RESEARC,10656815,2022,UNIVERSITY OF MIAMI CORAL GABLES,CORAL GABLES,331462926,FL,UNITED STATES,625174149,RQMFJGDTQ5V3


In [12]:
# remove the RADx fiscal year so we can use the fiscal year from the organizations dataframe instead.
pis.rename(columns={"fiscal_year": "fiscal_year_radx"}, inplace=True)

In [13]:
pi_orgs = pis.merge(organizations[["profileId", "organization.org_name", "organization.org_city", 
                         "organization.org_state", "organization.org_country",
                         "organization.primary_duns", "organization.primary_uei", "fiscal_year"]], on="profileId", how="left")
#pi_orgs.fillna("", inplace=True)

In [14]:
# add CURIES
pi_orgs["organization.primary_duns"] = pi_orgs["organization.primary_duns"].apply(grant_query.add_prefix, prefix="duns:")
pi_orgs["organization.primary_uei"] = pi_orgs["organization.primary_uei"].apply(grant_query.add_prefix, prefix="uei:")

In [15]:
pi_orgs.sort_values("fiscal_year", ascending=False, inplace=True)
pi_orgs.drop_duplicates("profileId", inplace=True)

In [16]:
pi_orgs["firstName"] = pi_orgs["firstName"].apply(grant_query.standardize_name)
pi_orgs["lastName"] = pi_orgs["lastName"].apply(grant_query.standardize_name)
pi_orgs["middleName"] = pi_orgs["middleName"].apply(grant_query.standardize_name)
pi_orgs["fullName"] = pi_orgs["fullName"].apply(grant_query.standardize_name)
pi_orgs["name"] = pi_orgs["lastName"] + " " + pi_orgs["firstName"].str[:1] + pi_orgs["middleName"].str[:1]

In [17]:
pi_orgs = pi_orgs[["profileId", "name", "firstName", "middleName", "lastName", "fullName", 
                   "coreProjectNum", "projectSerialNum", "fiscal_year", 
                   "organization.org_name", "organization.org_city",
                   "organization.org_state", "organization.org_country",
                   "organization.primary_duns", "organization.primary_uei"]]

In [18]:
pi_orgs.sort_values(["coreProjectNum"], ascending=False, inplace=True)

In [19]:
pi_orgs.to_csv("../derived_data/pi_organizations.csv", index=False)

In [20]:
print("Number of PIs:", pi_orgs.shape[0])
pi_orgs.head()

Number of PIs: 99


,profileId,name,firstName,middleName,lastName,fullName,coreProjectNum,projectSerialNum,fiscal_year,organization.org_name,organization.org_city,organization.org_state,organization.org_country,organization.primary_duns,organization.primary_uei
189,6570860,Bell D,Douglas,,Bell,Douglas Bell,ZIAES103366,ES103366,2023,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,None,None,UNITED STATES,,
161,1858811,Golan DE,David,E,Golan,David E Golan,U54HL119145,HL119145,2023,HARVARD MEDICAL SCHOOL,BOSTON,MA,UNITED STATES,duns:047006379,uei:JDLVAVGYJQ21
162,1876423,Loscalzo J,Joseph,,Loscalzo,Joseph Loscalzo,U54HL119145,HL119145,2024,BRIGHAM AND WOMEN'S HOSPITAL,BOSTON,MA,UNITED STATES,duns:030811269,uei:QN6MS4VN7BD1
163,8614812,Parrish JA,John,A,Parrish,John A Parrish,U54HL119145,HL119145,2022,MASSACHUSETTS GENERAL HOSPITAL,BOSTON,MA,UNITED STATES,duns:073130411,uei:FLJ7DQKLL226
18,9603016,Xu H,Hua,,Xu,Hua Xu,U24LM013755,LM013755,2021,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,HOUSTON,TX,UNITED STATES,duns:800771594,uei:ZUFBNVZ587D4


## Get ROR ids (manually curated list)

In [21]:
ror_ids = pd.read_csv(
        "https://docs.google.com/spreadsheets/d/1_7mYh75h15DCVSzD5ydQAEHsg6ljdKutQzSR7VvU-lE/export?format=csv",
        keep_default_na=False
       )

In [22]:
ror_ids.head()

,research_initiative,official_name,modified_name,original_name,ror,geonames
0,RADx-rad,National Institute of Environmental Health Sciences,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,00j4k1h63,4464368
1,RADx-rad,Massachusetts General Hospital,MASSACHUSETTS GENERAL HOSPITAL,MASSACHUSETTS GENERAL HOSPITAL,002pd6e78,4930956
2,RADx-rad,Harvard Medical School,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL,03wevmz92,4930956
3,RADx-rad,Brigham and Women's Hospital,BRIGHAM AND WOMEN'S HOSPITAL,BRIGHAM AND WOMEN'S HOSPITAL,04b6nzv94,4930956
4,RADx-rad,"University of California, San Diego","UNIVERSITY OF CALIFORNIA, SAN DIEGO","UNIVERSITY OF CALIFORNIA, SAN DIEGO",0168r3w48,5391811


In [23]:
# Add URLs and CURIEs
ror_ids["ror_url"] = ror_ids["ror"].apply(grant_query.add_prefix, prefix="https://ror.org/")
ror_ids["ror"] = ror_ids["ror"].apply(grant_query.add_prefix, prefix="ror:")

ror_ids["geonames_url"] = ror_ids["geonames"].apply(grant_query.add_prefix, prefix="https://www.geonames.org/")
ror_ids["geonames"] = ror_ids["geonames"].apply(grant_query.add_prefix, prefix="geonames:")

In [24]:
ror_ids

,research_initiative,official_name,modified_name,original_name,ror,geonames,ror_url,geonames_url
0,RADx-rad,National Institute of Environmental Health Sciences,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,ror:00j4k1h63,geonames:4464368,https://ror.org/00j4k1h63,https://www.geonames.org/4464368
1,RADx-rad,Massachusetts General Hospital,MASSACHUSETTS GENERAL HOSPITAL,MASSACHUSETTS GENERAL HOSPITAL,ror:002pd6e78,geonames:4930956,https://ror.org/002pd6e78,https://www.geonames.org/4930956
2,RADx-rad,Harvard Medical School,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL,ror:03wevmz92,geonames:4930956,https://ror.org/03wevmz92,https://www.geonames.org/4930956
3,RADx-rad,Brigham and Women's Hospital,BRIGHAM AND WOMEN'S HOSPITAL,BRIGHAM AND WOMEN'S HOSPITAL,ror:04b6nzv94,geonames:4930956,https://ror.org/04b6nzv94,https://www.geonames.org/4930956
4,RADx-rad,"University of California, San Diego","UNIVERSITY OF CALIFORNIA, SAN DIEGO","UNIVERSITY OF CALIFORNIA, SAN DIEGO",ror:0168r3w48,geonames:5391811,https://ror.org/0168r3w48,https://www.geonames.org/5391811
5,RADx-rad,Yale University,YALE UNIVERSITY,YALE UNIVERSITY,ror:03v76x132,geonames:4839366,https://ror.org/03v76x132,https://www.geonames.org/4839366
6,RADx-rad,The University of Texas Health Science Center at Houston,UNIVERSITY OF TEXAS HEALTH SCIENCE CENTER HOUSTON,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,ror:03gds6c39,geonames:4699066,https://ror.org/03gds6c39,https://www.geonames.org/4699066
7,RADx-rad,Virginia Commonwealth University,VIRGINIA COMMONWEALTH UNIVERSITY,VIRGINIA COMMONWEALTH UNIVERSITY,ror:02nkdxk79,geonames:4781708,https://ror.org/02nkdxk79,https://www.geonames.org/4781708
8,RADx-rad,University of Michigan–Ann Arbor,UNIVERSITY OF MICHIGAN ANN ARBOR,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ror:00jmfr291,geonames:4984247,https://ror.org/00jmfr291,https://www.geonames.org/4984247
9,RADx-rad,The Ohio State University,OHIO STATE UNIVERSITY,OHIO STATE UNIVERSITY,ror:00rs6vg23,geonames:4509177,https://ror.org/00rs6vg23,https://www.geonames.org/4509177


In [25]:
pi_orgs = pi_orgs.merge(ror_ids, left_on="organization.org_name", right_on="original_name", how="left")
pi_orgs.fillna("", inplace=True)
pi_orgs["profileId"] = "profileid:" + pi_orgs["profileId"]

In [26]:
# signin to sam.gov using login.gov and authenticate with Google code for pwrose.ucsd in Duo
# how to search:
# https://www.fsd.gov/gsafsd_sp?id=kb_article_view&sysparm_article=KB0041255&sys_kb_id=4f359fde1b59f1103565ed3ce54bcb92&spa=1
# DUNS number lookup: https://www.dnb.com/duns-number/lookup.html
# use ROR id if available, otherwise DUNS id, otherwise UEI

In [27]:
pi_orgs.head(10)

,profileId,name,firstName,middleName,lastName,fullName,coreProjectNum,projectSerialNum,fiscal_year,organization.org_name,organization.org_city,organization.org_state,organization.org_country,organization.primary_duns,organization.primary_uei,research_initiative,official_name,modified_name,original_name,ror,geonames,ror_url,geonames_url
0,profileid:6570860,Bell D,Douglas,,Bell,Douglas Bell,ZIAES103366,ES103366,2023,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,,,UNITED STATES,,,RADx-rad,National Institute of Environmental Health Sciences,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,ror:00j4k1h63,geonames:4464368,https://ror.org/00j4k1h63,https://www.geonames.org/4464368
1,profileid:1858811,Golan DE,David,E,Golan,David E Golan,U54HL119145,HL119145,2023,HARVARD MEDICAL SCHOOL,BOSTON,MA,UNITED STATES,duns:047006379,uei:JDLVAVGYJQ21,RADx-rad,Harvard Medical School,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL,ror:03wevmz92,geonames:4930956,https://ror.org/03wevmz92,https://www.geonames.org/4930956
2,profileid:1876423,Loscalzo J,Joseph,,Loscalzo,Joseph Loscalzo,U54HL119145,HL119145,2024,BRIGHAM AND WOMEN'S HOSPITAL,BOSTON,MA,UNITED STATES,duns:030811269,uei:QN6MS4VN7BD1,RADx-rad,Brigham and Women's Hospital,BRIGHAM AND WOMEN'S HOSPITAL,BRIGHAM AND WOMEN'S HOSPITAL,ror:04b6nzv94,geonames:4930956,https://ror.org/04b6nzv94,https://www.geonames.org/4930956
3,profileid:8614812,Parrish JA,John,A,Parrish,John A Parrish,U54HL119145,HL119145,2022,MASSACHUSETTS GENERAL HOSPITAL,BOSTON,MA,UNITED STATES,duns:073130411,uei:FLJ7DQKLL226,RADx-rad,Massachusetts General Hospital,MASSACHUSETTS GENERAL HOSPITAL,MASSACHUSETTS GENERAL HOSPITAL,ror:002pd6e78,geonames:4930956,https://ror.org/002pd6e78,https://www.geonames.org/4930956
4,profileid:9603016,Xu H,Hua,,Xu,Hua Xu,U24LM013755,LM013755,2021,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,HOUSTON,TX,UNITED STATES,duns:800771594,uei:ZUFBNVZ587D4,RADx-rad,The University of Texas Health Science Center at Houston,UNIVERSITY OF TEXAS HEALTH SCIENCE CENTER HOUSTON,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,ror:03gds6c39,geonames:4699066,https://ror.org/03gds6c39,https://www.geonames.org/4699066
5,profileid:1900244,Ohno-Machado L,Lucila,,Ohno-Machado,Lucila Ohno-Machado,U24LM013755,LM013755,2024,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,duns:043207562,uei:FL6GV84CKN57,RADx-rad,Yale University,YALE UNIVERSITY,YALE UNIVERSITY,ror:03v76x132,geonames:4839366,https://ror.org/03v76x132,https://www.geonames.org/4839366
6,profileid:10450719,Aronoff-Spencer ES,Eliah,S,Aronoff-Spencer,Eliah S Aronoff-Spencer,U24LM013755,LM013755,2017,"UNIVERSITY OF CALIFORNIA, SAN DIEGO",LA JOLLA,CA,UNITED STATES,duns:804355790,uei:UYTTZT6G9DT1,RADx-rad,"University of California, San Diego","UNIVERSITY OF CALIFORNIA, SAN DIEGO","UNIVERSITY OF CALIFORNIA, SAN DIEGO",ror:0168r3w48,geonames:5391811,https://ror.org/0168r3w48,https://www.geonames.org/5391811
7,profileid:8485057,Ward K,Kevin,,Ward,Kevin Ward,U18TR003812,TR003812,2009,VIRGINIA COMMONWEALTH UNIVERSITY,RICHMOND,VA,UNITED STATES,duns:105300446,uei:MLQFL4JSSAA9,RADx-rad,Virginia Commonwealth University,VIRGINIA COMMONWEALTH UNIVERSITY,VIRGINIA COMMONWEALTH UNIVERSITY,ror:02nkdxk79,geonames:4781708,https://ror.org/02nkdxk79,https://www.geonames.org/4781708
8,profileid:8485696,Fan X,Xudong,,Fan,Xudong Fan,U18TR003812,TR003812,2024,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES,duns:073133571,uei:GNJ7BBP73WE9,RADx-rad,University of Michigan–Ann Arbor,UNIVERSITY OF MICHIGAN ANN ARBOR,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ror:00jmfr291,geonames:4984247,https://ror.org/00jmfr291,https://www.geonames.org/4984247
9,profileid:12364332,Reategui E,Eduardo,,Reategui,Eduardo Reategui,U18TR003807,TR003807,2022,OHIO STATE UNIVERSITY,COLUMBUS,OH,UNITED STATES,duns:832127323,uei:DLWBSLWAJWR1,RADx-rad,The Ohio State University,OHIO STATE UNIVERSITY,OHIO STATE UNIVERSITY,ror:00rs6vg23,geonames:4509177,https://ror.org/00rs6vg

In [28]:
def get_preferred_organization_id(row):
    if row["ror"] != "":
        return row["ror"]
    if row["organization.primary_uei"] != "":
        return row["organization.primary_uei"]
    else:
        return row["organization.primary_duns"]

In [29]:
# Assign a unique organization id (preference ror > uei > duns)
pi_orgs["id"] = pi_orgs.apply(get_preferred_organization_id, axis=1)
pi_orgs.query("id != ''", inplace=True)

In [30]:
pi_orgs

,profileId,name,firstName,middleName,lastName,fullName,coreProjectNum,projectSerialNum,fiscal_year,organization.org_name,organization.org_city,organization.org_state,organization.org_country,organization.primary_duns,organization.primary_uei,research_initiative,official_name,modified_name,original_name,ror,geonames,ror_url,geonames_url,id
0,profileid:6570860,Bell D,Douglas,,Bell,Douglas Bell,ZIAES103366,ES103366,2023,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,,,UNITED STATES,,,RADx-rad,National Institute of Environmental Health Sciences,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,NATIONAL INSTITUTE OF ENVIRONMENTAL HEALTH SCIENCES,ror:00j4k1h63,geonames:4464368,https://ror.org/00j4k1h63,https://www.geonames.org/4464368,ror:00j4k1h63
1,profileid:1858811,Golan DE,David,E,Golan,David E Golan,U54HL119145,HL119145,2023,HARVARD MEDICAL SCHOOL,BOSTON,MA,UNITED STATES,duns:047006379,uei:JDLVAVGYJQ21,RADx-rad,Harvard Medical School,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL,ror:03wevmz92,geonames:4930956,https://ror.org/03wevmz92,https://www.geonames.org/4930956,ror:03wevmz92
2,profileid:1876423,Loscalzo J,Joseph,,Loscalzo,Joseph Loscalzo,U54HL119145,HL119145,2024,BRIGHAM AND WOMEN'S HOSPITAL,BOSTON,MA,UNITED STATES,duns:030811269,uei:QN6MS4VN7BD1,RADx-rad,Brigham and Women's Hospital,BRIGHAM AND WOMEN'S HOSPITAL,BRIGHAM AND WOMEN'S HOSPITAL,ror:04b6nzv94,geonames:4930956,https://ror.org/04b6nzv94,https://www.geonames.org/4930956,ror:04b6nzv94
3,profileid:8614812,Parrish JA,John,A,Parrish,John A Parrish,U54HL119145,HL119145,2022,MASSACHUSETTS GENERAL HOSPITAL,BOSTON,MA,UNITED STATES,duns:073130411,uei:FLJ7DQKLL226,RADx-rad,Massachusetts General Hospital,MASSACHUSETTS GENERAL HOSPITAL,MASSACHUSETTS GENERAL HOSPITAL,ror:002pd6e78,geonames:4930956,https://ror.org/002pd6e78,https://www.geonames.org/4930956,ror:002pd6e78
4,profileid:9603016,Xu H,Hua,,Xu,Hua Xu,U24LM013755,LM013755,2021,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,HOUSTON,TX,UNITED STATES,duns:800771594,uei:ZUFBNVZ587D4,RADx-rad,The University of Texas Health Science Center at Houston,UNIVERSITY OF TEXAS HEALTH SCIENCE CENTER HOUSTON,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,ror:03gds6c39,geonames:4699066,https://ror.org/03gds6c39,https://www.geonames.org/4699066,ror:03gds6c39
5,profileid:1900244,Ohno-Machado L,Lucila,,Ohno-Machado,Lucila Ohno-Machado,U24LM013755,LM013755,2024,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,duns:043207562,uei:FL6GV84CKN57,RADx-rad,Yale University,YALE UNIVERSITY,YALE UNIVERSITY,ror:03v76x132,geonames:4839366,https://ror.org/03v76x132,https://www.geonames.org/4839366,ror:03v76x132
6,profileid:10450719,Aronoff-Spencer ES,Eliah,S,Aronoff-Spencer,Eliah S Aronoff-Spencer,U24LM013755,LM013755,2017,"UNIVERSITY OF CALIFORNIA, SAN DIEGO",LA JOLLA,CA,UNITED STATES,duns:804355790,uei:UYTTZT6G9DT1,RADx-rad,"University of California, San Diego","UNIVERSITY OF CALIFORNIA, SAN DIEGO","UNIVERSITY OF CALIFORNIA, SAN DIEGO",ror:0168r3w48,geonames:5391811,https://ror.org/0168r3w48,https://www.geonames.org/5391811,ror:0168r3w48
7,profileid:8485057,Ward K,Kevin,,Ward,Kevin Ward,U18TR003812,TR003812,2009,VIRGINIA COMMONWEALTH UNIVERSITY,RICHMOND,VA,UNITED STATES,duns:105300446,uei:MLQFL4JSSAA9,RADx-rad,Virginia Commonwealth University,VIRGINIA COMMONWEALTH UNIVERSITY,VIRGINIA COMMONWEALTH UNIVERSITY,ror:02nkdxk79,geonames:4781708,https://ror.org/02nkdxk79,https://www.geonames.org/4781708,ror:02nkdxk79
8,profileid:8485696,Fan X,Xudong,,Fan,Xudong Fan,U18TR003812,TR003812,2024,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES,duns:073133571,uei:GNJ7BBP73WE9,RADx-rad,University of Michigan–Ann Arbor,UNIVERSITY OF MICHIGAN ANN ARBOR,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ror:00jmfr291,geonames:4984247,https://ror.org/00jmfr291,https://www.geonames.org/4984247,ror:00jmfr291
9,profileid:12364332,Reategui E,Eduardo,,Reategui,Eduardo Reategui,U18TR003807,TR003807,2022,OHIO STATE UNIVERSITY,COLUMBUS,OH,UNITED STATES,duns:832127323,uei:DLWBSLWAJWR1,RADx

In [31]:
pi_orgs.query("id == 'uei:H6D6JMXJXDE6'")

,profileId,name,firstName,middleName,lastName,fullName,coreProjectNum,projectSerialNum,fiscal_year,organization.org_name,organization.org_city,organization.org_state,organization.org_country,organization.primary_duns,organization.primary_uei,research_initiative,official_name,modified_name,original_name,ror,geonames,ror_url,geonames_url,id
69,profileid:6786579,Salazar JC,Juan,C,Salazar,Juan C Salazar,R61HD105613,HD105613,2023,UNIVERSITY OF CONNECTICUT SCH OF MED/DNT,FARMINGTON,CT,UNITED STATES,duns:022254226,uei:H6D6JMXJXDE6,,,,,,,,,uei:H6D6JMXJXDE6


### Create Organization nodes

In [32]:
# search Unique Entity ID (UEI) see: https://www.fsd.gov/gsafsd_sp?id=kb_article_view&sysparm_article=KB0041255&sys_kb_id=4f359fde1b59f1103565ed3ce54bcb92&spa=1
organization_map = {"id": "id", "official_name": "name", 
                    "organization.org_city": "city", "organization.org_state": "state", "organization.org_country": "country", 
                    "ror": "ror", "organization.primary_uei": "uei",
                    "organization.primary_duns": "duns", "geonames": "geonames", "ror_url": "url"}

organization_nodes = pi_orgs[organization_map.keys()].copy()
organization_nodes.rename(columns=organization_map, inplace=True)
organization_nodes.dropna(inplace=True)
organization_nodes.drop_duplicates(subset="id", inplace=True)

In [33]:
# TODO: there are two entries without a name ???

In [34]:
organization_nodes.to_csv(os.path.join(KG_PATH, "nodes", "Organization.csv"), index=False)

In [35]:
organization_nodes.columns

Index(['id', 'name', 'city', 'state', 'country', 'ror', 'uei', 'duns',
       'geonames', 'url'],
      dtype='object')

### Create Researcher-EMPLOYED_AT-Organization relationships

In [36]:
employed_at_map = {"profileId": "from", "id": "to", "fiscal_year": "lastUpdated"}

In [37]:
employed_at = pi_orgs[employed_at_map.keys()].copy()
employed_at.rename(columns=employed_at_map, inplace=True)
employed_at = employed_at[(employed_at["from"] != "") & (employed_at["to"] != "")]
employed_at.dropna(inplace=True)
employed_at.drop_duplicates(inplace=True)

In [38]:
employed_at.to_csv(os.path.join(KG_PATH, "relationships", "Researcher-EMPLOYED_AT-Organization.csv"), index=False)

In [39]:
employed_at.head()

,from,to,lastUpdated
0,profileid:6570860,ror:00j4k1h63,2023
1,profileid:1858811,ror:03wevmz92,2023
2,profileid:1876423,ror:04b6nzv94,2024
3,profileid:8614812,ror:002pd6e78,2022
4,profileid:9603016,ror:03gds6c39,2021
